<h1 align=center>MIT Covid-19 Datathon</h1>

In [1]:
# Standard Tool Imports
# Use pandas for dataframe tools
import pandas as pd

In [2]:
# These definitions are run time environment specific
data_path = '/home/michael/Documents/Data/safegraph/v1/main-file/'
backfill_path = '/home/michael/Documents/Data/safegraph/patterns_backfill/2020/05/07/12/2019/'
sub_dirs = ['01/', '02/', '03/', '04/', '05/', '06/', '07/', '08/', '09/', '10/', '11/', '12/']
#sub_dirs = ['01/', '02/', '03/', '04/']
backfill_files = ['patterns-part1.csv', 'patterns-part2.csv', 'patterns-part3.csv', 'patterns-part4.csv']

In [3]:
# Define the files we want to process
visit_data_files = ['2020-04-26-weekly-patterns.csv']

core_poi_data_files = ['core_poi-part1.csv', 'core_poi-part2.csv','core_poi-part3.csv',\
                      'core_poi-part4.csv', 'core_poi-part5.csv']

In [4]:
# Define our operating parameters
focus_states = ['RI', 'CT']

# Define NCIS Codes to save

# Pattern file columns to keep
pattern_keep_columns = ['safegraph_place_id', 'location_name', 'region', 'date_range_start', 'date_range_end', \
                        'postal_code',  'raw_visit_counts', 'median_dwell', 'bucketed_dwell_times']

core_poi_keep_columns = ['safegraph_place_id', 'latitude', 'longitude', 'naics_code']


In [5]:
# This function will split the bucketed dwell field and calculate an estimated mean
def calc_dwells(dwell_col) :
    total_dwell_list = []
    dwell_buckets = {}
    bucket_multipliers = {'<5' : 2.5, '5-20' : 12.5, '21-60' : 40.5, '61-240' : 150.5, '>240' : 240}
    
    # run through each row in the series
    
    for row in dwell_col :
        # ignore the starting and ending brackets
        entries = row[1:-1].split(',')

        # for each bucket, split the key from the value
        for entry in entries:
            bucket, count = entry.split(':')
            dwell_buckets[bucket.replace('\"', '')] = int(count)


            total_dwell = 0
            
            # Now use the dictionary to calculate the total dwell time
            for bucket in dwell_buckets :
                total_dwell += dwell_buckets[bucket] * bucket_multipliers[bucket]

        total_dwell_list.append(total_dwell)
    
    # return the list of total dwell times
    return(total_dwell_list)

In [6]:
loc_df = pd.read_csv('placeCountyCBG.csv')

In [7]:
# This series of steps creates a dataframe with all the core poi entries

# initialize the data frame
big_core_df = pd.read_csv(data_path+core_poi_data_files[0])

# only keep the columns we'll use in our analysis
big_core_df = big_core_df[core_poi_keep_columns]

# Run through all the other files in the list and append
for file_name in core_poi_data_files[1:]:
    try:
        # read the next data file and concat on the end
        df2 = pd.read_csv(data_path+file_name)
        df2 = df2[core_poi_keep_columns]
        big_core_df = pd.concat([big_core_df, df2])
        
        # Force the deletion of df2 to save memory
        del df2
    except OSError :
        print("Unable to open file {}".format(data_path+file_name))
        continue

In [10]:
# The safegraph data is stored in a subdirectory for each month
for sd in sub_dirs:
    idx = 0;
    
    # Each month is divided into four files with standard names
    for file_name in backfill_files:
        idx += 1;
        
        # Read the first datafile into a dataframe
        df = pd.read_csv(backfill_path+sd+file_name)
        
        # only keep the columns of interest
        df = df[pattern_keep_columns]
        
        # Calculate the total dwell from the buckets
        df['est_total_dwell'] = calc_dwells(df['bucketed_dwell_times'])
        
        # Merge the file with the core poi info and the location information
        big_df = df.merge(big_core_df, how='left')
        big_df = big_df.merge(loc_df, how='left')

        # Filter the dataframe by state and save a state specific data file
        for state in focus_states :
            new_big_df = big_df[big_df['region'] == state]
            new_file_name = 'backfill-patterns'+str(idx)+'-'+sd[:-1]+'-2019-'+state+'.csv'
            new_big_df.to_csv(new_file_name)
            print(new_file_name)

backfill-patterns1-01-2019-RI.csv
backfill-patterns1-01-2019-CT.csv
backfill-patterns2-01-2019-RI.csv
backfill-patterns2-01-2019-CT.csv
backfill-patterns3-01-2019-RI.csv
backfill-patterns3-01-2019-CT.csv
backfill-patterns4-01-2019-RI.csv
backfill-patterns4-01-2019-CT.csv
backfill-patterns1-02-2019-RI.csv
backfill-patterns1-02-2019-CT.csv
backfill-patterns2-02-2019-RI.csv
backfill-patterns2-02-2019-CT.csv
backfill-patterns3-02-2019-RI.csv
backfill-patterns3-02-2019-CT.csv
backfill-patterns4-02-2019-RI.csv
backfill-patterns4-02-2019-CT.csv
backfill-patterns1-03-2019-RI.csv
backfill-patterns1-03-2019-CT.csv
backfill-patterns2-03-2019-RI.csv
backfill-patterns2-03-2019-CT.csv
backfill-patterns3-03-2019-RI.csv
backfill-patterns3-03-2019-CT.csv
backfill-patterns4-03-2019-RI.csv
backfill-patterns4-03-2019-CT.csv
backfill-patterns1-04-2019-RI.csv
backfill-patterns1-04-2019-CT.csv
backfill-patterns2-04-2019-RI.csv
backfill-patterns2-04-2019-CT.csv
backfill-patterns3-04-2019-RI.csv
backfill-patte